## Train ResNets

### created by Yuying Liu, 04/30/2020

This script is a template for training neural network time-steppers for different systems and different time scales. To reproduce the results in the paper, one needs to obtain all 11 neural network models for each nonlinear system under study. For setup details, please refer to Table 2 in the paper.

In [8]:
import os
import sys
import torch
import numpy as np

module_path = os.path.abspath(os.path.join('../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import ResNet as net

In [9]:
# adjustables

k = 10                        # model index: should be in {0, 2, ..., 10}
dt = 0.01                     # time unit: 0.0005 for Lorenz and 0.01 for others
system = 'Hyperbolic'         # system name: 'Hyperbolic', 'Cubic', 'VanDerPol', 'Hopf' or 'Lorenz'
noise = 0.0                   # noise percentage: 0.00, 0.01 or 0.02

lr = 1e-3                     # learning rate
max_epoch = 500000            # the maximum training epoch 
batch_size = 320              # training batch size
arch = [2, 128, 128, 128, 2]  # architecture of the neural network

In [10]:
# paths
data_dir = os.path.join('data/', system)
model_dir = os.path.join('models/', system)

# global const
n_forward = 5
step_size = 2**k

In [11]:
# load data
train_data = np.load(os.path.join(data_dir, 'train_noise{}.npy'.format(noise)))
val_data = np.load(os.path.join(data_dir, 'val_noise{}.npy'.format(noise)))
test_data = np.load(os.path.join(data_dir, 'test_noise{}.npy'.format(noise)))
n_train = train_data.shape[0]
n_val = val_data.shape[0]
n_test = test_data.shape[0]

# create dataset object
dataset = net.DataSet(train_data, val_data, test_data, dt, step_size, n_forward)

In [12]:
model_name = 'model_D{}_noise{}.pt'.format(step_size, noise)

# create/load model object
try:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = torch.load(os.path.join(model_dir, model_name), map_location=device)
    model.device = device
except:
    print('create model {} ...'.format(model_name))
    model = net.ResNet(arch=arch, dt=dt, step_size=step_size)

# training
model.train_net(dataset, max_epoch=max_epoch, batch_size=batch_size, lr=lr,
                model_path=os.path.join(model_dir, model_name))

579981055e-08
epoch 281000, training loss 1.7632535787015513e-07, validation loss 2.2838884206066723e-07
epoch 282000, training loss 3.3617300942978545e-08, validation loss 7.235792054416379e-08
epoch 283000, training loss 3.3195138371411304e-07, validation loss 3.9795136785869545e-07
epoch 284000, training loss 6.111683319431904e-07, validation loss 6.811233106418513e-07
epoch 285000, training loss 3.559140537845451e-08, validation loss 7.312746674870141e-08
epoch 286000, training loss 1.4560803229812791e-08, validation loss 5.817731718593677e-08
epoch 287000, training loss 6.714669353868885e-08, validation loss 1.205312400998082e-07
epoch 288000, training loss 1.1454019244183655e-07, validation loss 1.5219545446143457e-07
epoch 289000, training loss 2.58922995044486e-07, validation loss 3.0776507742302783e-07
epoch 290000, training loss 3.6762056510042385e-08, validation loss 7.669358126349834e-08
epoch 291000, training loss 2.7060215401775167e-08, validation loss 6.715009703839314e-